<a href="https://colab.research.google.com/github/instrat-pl/pypsa-pl-mini/blob/main/notebooks/pypsa_pl_mini_opex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyPSA-PL-mini: OPEX

Simplified energy modelling environment for rapid testing and education.

Example of OPEX optimisation.

Version 0.1

This notebook is released under [CC-BY-4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact person:** Patryk Kubiczek (patryk.kubiczek@instrat.pl)

## How to use it?

**Start by making your own copy of this notebook (File > Save a copy in Drive).** This notebook, containing an application of the the PyPSA-PL-mini model, is synchronised with the GitHub repository https://github.com/instrat-pl/pypsa-pl-mini. To play with the model, create your own copy of this notebook. 

[*General user*] 
Run the "Configuration" cells which will clone the PyPSA-PL-mini repository into your Google Colab space and which will install all the required libraries. This might take up to a few minutes. After the configuration is finished, you can proceed to experiment with PyPSA-PL-mini. Have fun!

## Configuration (run each cell just once)

In [1]:
import sys
import os

# Optionally install pl_PL.UTF-8 locale in Google Colab
# Source: https://stackoverflow.com/questions/67045349/change-locale-for-google-colab

skip_installing_pl_locale = True

if "google.colab" in sys.modules and not skip_installing_pl_locale:
  # Install pl_PL
  !/usr/share/locales/install-language-pack pl_PL.UTF-8
  !dpkg-reconfigure locales
    
  # Restart Python process to pick up the new locales
  os.kill(os.getpid(), 9)

In [2]:
import sys
import os
from pathlib import Path

instrat_user = False
force_installation = False

if "google.colab" in sys.modules:

  %cd "/content"

  if instrat_user:  
    
    from google.colab import drive
    root = "/content/drive"
    drive.mount(root)

    def project_dir(*path):
      return Path(root, "MyDrive", "Colab", "PyPSA-PL-mini", *path)

  else:
    
    from google.colab import userdata
    # ghtoken = userdata.get("GHTOKEN")

    !rm -rf pypsa-pl-mini
    # !git clone https://{ghtoken}@github.com/instrat-pl/pypsa-pl-mini.git
    !git clone https://github.com/instrat-pl/pypsa-pl-mini.git

    def project_dir(*path):
      return Path("/content", "pypsa-pl-mini", *path)

  %cd {str(project_dir())}

  if not instrat_user or force_installation:
    !pip install poetry --quiet
    !poetry config virtualenvs.in-project false
    !poetry config virtualenvs.path {str(project_dir("venv"))}
    !poetry install --no-ansi
    # ipywidgets have to be downgraded in Google Colab
    !poetry add ipywidgets@7.7.2 --no-ansi
    # Download and install Work Sans font
    !mkdir fonts
    %cd fonts  
    !wget https://github.com/weiweihuanghuang/Work-Sans/raw/master/fonts/ttf/WorkSans-Regular.ttf
    !wget https://github.com/weiweihuanghuang/Work-Sans/raw/master/fonts/ttf/WorkSans-Medium.ttf
    from matplotlib import font_manager
    font_files = ["WorkSans-Regular.ttf", "WorkSans-Medium.ttf"]
    for font_file in font_files:
        font_manager.fontManager.addfont(font_file)
    %cd ..

  v = f"{sys.version_info.major}.{sys.version_info.minor}"
  venv_location = !poetry env info -p
  VENV_PATH = os.path.join(venv_location[0], "lib", f"python{v}", "site-packages")
  sys.path.insert(0, VENV_PATH)  
  
  SRC_PATH = str(project_dir("src"))
  sys.path.insert(0, SRC_PATH)

else:
  %load_ext autoreload
  %autoreload 2

import pypsa_pl_mini.config

## Power system model

### Specify parameters

In [3]:
params = {
    # Run name and year
    "run_name": "pypsa_pl_mini_opex",
    "year": 2023,
    # Input data
    "technology_carrier_definitions": "mini",
    "technology_cost_data": "instrat_2024",
    "installed_capacity": "historical_totals",
    "annual_energy_flows": "historical",
    "capacity_utilisation": "historical",
    # Other assumptions
    "investment_technologies": [],
    "retirement_technologies": [],
    "constrained_energy_flows": [],
    "weather_year": 2012,
    "share_space_heating": 0.8,
    "prosumer_self_consumption": 0.2,
    "discount_rate": 0.03, # irrelevant in this example
    "reoptimise_with_fixed_capacities": False, # capacities are always fixed in this example
    # Technical details
    "inf": 100000,
    "reverse_links": True,
    "solver": "highs",
}

### Prepare inputs


In [4]:
from pypsa_pl_mini.build_network import load_and_preprocess_inputs


def custom_operation(inputs):

    def add_qualifier_to_technology(df, technology, qualifier):
        df.loc[df["technology"] == technology, "qualifier"] = qualifier
        return df

    # Identify solar PV roof as prosumer electricity source
    inputs["installed_capacity"] = add_qualifier_to_technology(
        inputs["installed_capacity"],  "solar PV roof", "prosumer",
    )

    
    def remove_capacities(df, technologies):
        df = df[~df["technology"].isin(technologies)]
        return df

    # We do not model cross border electricity flows in this simplified example
    inputs["installed_capacity"] = remove_capacities(
        inputs["installed_capacity"],
        technologies=[
            "electricity export AC",
            "electricity import AC",
            "electricity export DC",
            "electricity import DC",
        ],
    )

    return inputs


inputs = load_and_preprocess_inputs(params, custom_operation=custom_operation)

2024-06-26 17:10:18 [INFO] NumExpr defaulting to 8 threads.


In [5]:
# name = "technology_carrier_definitions"
# name = "technology_cost_data"
# name = "installed_capacity"
# name = "annual_energy_flows"
# name = "capacity_utilisation"
# name = "co2_cost"
name = "final_use"

inputs[name]

parameter,area,carrier,year,p_set_annual
0,PL,electricity final use,2022,18310.502283
1,PL,electricity final use,2023,17614.155251


### Create PyPSA network

In [6]:
from pypsa_pl_mini.build_network import create_custom_network

network = create_custom_network(params)

network

Empty PyPSA Network 'pypsa_pl_mini_opex'
Components: none
Snapshots: 1

### Add snapshots

In [7]:
from pypsa_pl_mini.build_network import add_snapshots

add_snapshots(network, params)

network.snapshots

Index(['2012-03-09 00:00:00', '2012-03-09 01:00:00', '2012-03-09 02:00:00',
       '2012-03-09 03:00:00', '2012-03-09 04:00:00', '2012-03-09 05:00:00',
       '2012-03-09 06:00:00', '2012-03-09 07:00:00', '2012-03-09 08:00:00',
       '2012-03-09 09:00:00',
       ...
       '2012-11-01 14:00:00', '2012-11-01 15:00:00', '2012-11-01 16:00:00',
       '2012-11-01 17:00:00', '2012-11-01 18:00:00', '2012-11-01 19:00:00',
       '2012-11-01 20:00:00', '2012-11-01 21:00:00', '2012-11-01 22:00:00',
       '2012-11-01 23:00:00'],
      dtype='object', name='snapshot', length=672)

### Add carriers

In [8]:
from pypsa_pl_mini.build_network import add_carriers

add_carriers(network, inputs, params)

network.carriers

,color,order,aggregation,co2_emissions,nice_name,max_growth,max_relative_growth
Carrier,,,,,,,
DSR reduction,#1b1c1e,191,DSR,NaN,,inf,0.0
battery large charger,#458aff,142,battery large,NaN,,inf,0.0
battery large power,#458aff,141,battery large,NaN,,inf,0.0
battery large storage,#458aff,140,battery large,NaN,,inf,0.0
biogas CHP,#816d66,82,biomass and biogas CHP,NaN,,inf,0.0
biogas production,#816d66,81,biomass and biogas supply,NaN,,inf,0.0
biogas substrate supply,#816d66,80,biomass and biogas supply,NaN,,inf,0.0
biomass agriculture CHP,#ab9d99,71,biomass and biogas CHP,NaN,,inf,0.0
biomass agriculture supply,#ab9d99,70,biomass and biogas supply,NaN,,inf,0.0


### Add buses

In [9]:
from pypsa_pl_mini.build_network import add_buses

add_buses(network, inputs, params)

network.buses

,area,carrier,v_nom,type,x,y,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network
Bus,,,,,,,,,,,,,
PL battery large electricity,PL,battery large electricity,1.0,,0.0,0.0,,1.0,0.0,inf,PQ,,
PL biogas,PL,biogas,1.0,,0.0,0.0,,1.0,0.0,inf,PQ,,
PL biogas substrate,PL,biogas substrate,1.0,,0.0,0.0,,1.0,0.0,inf,PQ,,
PL biomass wood,PL,biomass wood,1.0,,0.0,0.0,,1.0,0.0,inf,PQ,,
PL electricity,PL,electricity,1.0,,0.0,0.0,,1.0,0.0,inf,PQ,,
PL hard coal,PL,hard coal,1.0,,0.0,0.0,,1.0,0.0,inf,PQ,,
PL hydro PSH electricity,PL,hydro PSH electricity,1.0,,0.0,0.0,,1.0,0.0,inf,PQ,,
PL lignite,PL,lignite,1.0,,0.0,0.0,,1.0,0.0,inf,PQ,,
PL natural gas,PL,natural gas,1.0,,0.0,0.0,,1.0,0.0,inf,PQ,,


### Add generating, consuming, and storing capacity (generators, links, stores)

#### Process installed capacity data and specify the relevant attributes

In [10]:
from pypsa_pl_mini.build_network import process_capacity_data

df_cap = process_capacity_data(inputs, params)

df_cap.head()

df_cap

,name,area,technology,qualifier,build_year,retire_year,cumulative,nom,carrier,aggregation,...,p_nom_extendable,p_nom_min,p_nom_max,e_nom_extendable,e_nom_min,e_nom_max,p_set_annual,p_min_pu,p_max_pu,p_set
1,PL battery large charger 2023,PL,battery large charger,NaN,2023,2023,True,140.0,battery large charger,battery large,...,False,140.0,140.0,NaN,NaN,NaN,NaN,0.00,1.0,NaN
0,PL battery large power 2023,PL,battery large power,NaN,2023,2023,True,150.0,battery large power,battery large,...,False,150.0,150.0,NaN,NaN,NaN,NaN,0.00,1.0,NaN
2,PL battery large storage 2023,PL,battery large storage,NaN,2023,2023,True,600.0,battery large storage,battery large,...,NaN,NaN,NaN,False,600.0,600.0,NaN,0.00,1.0,NaN
3,PL biogas CHP industrial 2023,PL,biogas CHP,industrial,2023,2023,True,70.0,biogas CHP,biomass and biogas CHP,...,False,70.0,70.0,NaN,NaN,NaN,NaN,0.00,1.0,34.3
4,PL biogas CHP public 2023,PL,biogas CHP,public,2023,2023,True,210.0,biogas CHP,biomass and biogas CHP,...,False,210.0,210.0,NaN,NaN,NaN,113.400000,0.00,1.0,NaN
5,PL biogas production industrial 2023,PL,biogas production,industrial,2023,2023,True,180.0,biogas production,biomass and biogas supply,...,False,180.0,180.0,NaN,NaN,NaN,NaN,0.00,1.0,88.2
6,PL biogas production public 2023,PL,biogas production,public,2023,2023,True,520.0,biogas production,biomass and biogas supply,...,False,520.0,520.0,NaN,NaN,NaN,NaN,0.00,1.0,NaN
7,PL biogas substrate supply 2023,PL,biogas substrate supply,NaN,2023,2023,True,100000.0,biogas substrate supply,biomass and biogas supply,...,False,100000.0,100000.0,NaN,NaN,NaN,NaN,0.00,1.0,NaN
8,PL biomass wood CHP industrial 2023,PL,biomass wood CHP,industrial,2023,2023,True,200.0,biomass wood CHP,biomass and biogas CHP,...,False,200.0,200.0,NaN,NaN,NaN,NaN,0.00,1.0,182.0
9,PL biomass wood CHP public 2023,PL,biomass wood CHP,public,2023,2023,True,260.0,biomass wood CHP,biomass and biogas CHP,...,False,260.0,260.0,NaN,NaN,NaN,228.800000,0.00,1.0,NaN


#### Specify which attributes are time dependent

In [11]:
from pypsa_pl_mini.define_time_dependent_attributes import (
    define_time_dependent_attributes,
)


df_attr_t = define_time_dependent_attributes(df_cap, params)

df_attr_t

,carrier,qualifier,attribute,profile_type
0,electricity final use,none,p_set,electricity final use load profile
1,biogas CHP,public,p_set,heat final use load profile
2,biomass wood CHP,public,p_set,heat final use load profile
3,hard coal CHP,public,p_set,heat final use load profile
4,natural gas CHP,public,p_set,heat final use load profile
5,solar PV ground,none,p_max_pu,vres availability profile
6,solar PV roof,prosumer,p_max_pu,vres availability profile
7,wind onshore,none,p_max_pu,vres availability profile
8,biogas CHP,industrial,p_set,constant load profile
9,biogas production,industrial,p_set,constant load profile


#### Create actual components and fill them with data

In [ ]:
from pypsa_pl_mini.build_network import add_capacities

add_capacities(network, df_cap, df_attr_t, params)

In [ ]:
network.generators.head()

In [ ]:
network.generators_t.p_max_pu.head()

In [ ]:
network.generators_t.p_set.head()

In [ ]:
network.links.head()

In [ ]:
network.links_t.p_set.head()

In [ ]:
network.links.p_max_pu

In [ ]:
network.stores.head()

### Add flow constraints

In [ ]:
from pypsa_pl_mini.build_network import add_energy_flow_constraints

add_energy_flow_constraints(network, inputs, params)

# No flow constraints are added by default
network.global_constraints

### Save input network

In [ ]:
from pypsa_pl_mini.config import data_dir

os.makedirs(data_dir("runs", params["run_name"]), exist_ok=True)
network.export_to_csv_folder(data_dir("runs", params["run_name"], "input_network"))

### Solve the model

In [ ]:
from pypsa_pl_mini.optimise_network import optimise_network

optimise_network(network, params)

### Save output network

In [ ]:
network.export_to_csv_folder(data_dir("runs", params["run_name"], "output_network"))

### Analyse results

In [ ]:
import pandas as pd


def append_annual_sum(df, value_cols=["value"]):
    agg = "carrier" if "carrier" in df.columns else "aggregation"
    return pd.concat(
        [
            df,
            df.groupby("year")
            .agg({agg: lambda x: "SUM", **{col: "sum" for col in value_cols}})
            .reset_index(),
        ]
    )

#### Display statistics

In [ ]:
from pypsa_pl_mini.process_output_network import calculate_statistics

df_stat = calculate_statistics(network)
df_stat

In [ ]:
# Example of statistics use: curtailed vRES energy ratio
df = df_stat.groupby("carrier")[["Supply", "Curtailment"]].sum()
df = 1 / (1 + df["Supply"] / df["Curtailment"])
df = df[df > 0].round(3).rename("value").to_frame()
df

#### Plot electrical capacity mix

In [ ]:
from pypsa_pl_mini.plot_outputs import plot_installed_capacities

fig, df = plot_installed_capacities(network)

df = append_annual_sum(df)
df

#### Plot electricity generation mix

In [ ]:
from pypsa_pl_mini.plot_outputs import plot_annual_generation

fig, df = plot_annual_generation(network)

df = append_annual_sum(df)
df

#### Plot fuel consumption and CO2 emissions

In [ ]:
from pypsa_pl_mini.plot_outputs import plot_fuel_consumption

fig, df = plot_fuel_consumption(network)

df = append_annual_sum(df)
df

In [ ]:
from pypsa_pl_mini.plot_outputs import plot_co2_emissions

fig, df = plot_co2_emissions(network)

df = append_annual_sum(df)
df

#### Plot hourly electricity generation

In [ ]:
from pypsa_pl_mini.plot_outputs import plot_hourly_generation

n_per_subperiod = 7 * 24
subperiods = [
    (subperiod, (i * n_per_subperiod, (i + 1) * n_per_subperiod))
    for i, subperiod in enumerate(["Jan-Mar", "Apr-Jun", "Aug-Sep", "Oct-Dec"])
]

fig, df = plot_hourly_generation(network, subperiods=subperiods, ylim=(-6,26))

In [ ]:
df

#### Plot marginal cost of electricity production

In [ ]:
from pypsa_pl_mini.plot_outputs import plot_prices

n_per_subperiod = 7 * 24
subperiods = [
    (subperiod, (i * n_per_subperiod, (i + 1) * n_per_subperiod))
    for i, subperiod in enumerate(["Jan-Mar", "Apr-Jun", "Aug-Sep", "Oct-Dec"])
]

fig, df = plot_prices(network, subperiods=subperiods, ylim=(0, 850))

df_price = df.copy()

In [ ]:
from pypsa_pl_mini.plot_outputs import plot_average_unit_cost_and_price

fig, df = plot_average_unit_cost_and_price(network)

# TGE-DA 2023 mean: 530 PLN/MWh

# Calculate simple (i.e. non-weighted) avg. market price
mean_price = df_price.mean().values[0].round(1)
print("Avg. electricity price (simple):", mean_price, "PLN/MWh")

df

#### Plot operational costs

In [ ]:
from pypsa_pl_mini.plot_outputs import plot_opex

fig, df = plot_opex(network)

# The sum of OPEX is the objective function's value
df = append_annual_sum(df)

df

#### Plot revenues net costs

In [ ]:
from pypsa_pl_mini.plot_outputs import plot_net_revenues

fig, df = plot_net_revenues(network, costs=["OPEX"])

df = append_annual_sum(df)

df

# CHP plants have negative net revenues as we do not take into account revenues from heat sales

### Sensitivity analysis: conventional baseload

In [ ]:
import logging
from matplotlib import pyplot as plt
from io import BytesIO
import ipywidgets as widgets
from ipywidgets import interact_manual

from pypsa_pl_mini.helper_functions import suppress_stdout

In [ ]:
default_p_min_pu = 0.25

def update_p_min_pu_of_public_power_plants(network, p_min_pu=default_p_min_pu):
    is_public_pp = network.links["carrier"].isin(
        ["hard coal power", "lignite power", "biomass wood power"]
    ) & (network.links["qualifier"] == "public")
    if network.meta["reverse_links"]:
        network.links.loc[is_public_pp, "p_max_pu"] = -p_min_pu
    else:
        network.links.loc[is_public_pp, "p_min_pu"] = p_min_pu
    return network

In [ ]:
from pypsa_pl_mini.plot_outputs import plot_curtailed_vres_energy

n_per_subperiod = 7 * 24
subperiods = [
    (subperiod, (i * n_per_subperiod, (i + 1) * n_per_subperiod))
    for i, subperiod in enumerate(["Jan-Mar", "Apr-Jun", "Aug-Sep", "Oct-Dec"])
]

figure_width = 6

def list_plotting_tasks():
    return [
        (
            plot_hourly_generation,
            dict(subperiods=[subperiods[1]], ylim=(-6, 26), figsize=(figure_width, 6)),
        ),
        (
            plot_prices,
            dict(subperiods=[subperiods[1]], ylim=(0, 850), figsize=(figure_width, 4)),
        ),
        (
            plot_annual_generation,
            dict(ylim=(-11, 171), figsize=(figure_width, 6)),
        ),
        (
            plot_curtailed_vres_energy,
            dict(ylim=(0, 5.5), figsize=(figure_width, 4)),
        ),
        (
            plot_co2_emissions,
            dict(ylim=(0, 125), figsize=(figure_width, 5)),
        ),
        (
            plot_opex,
            dict(ylim=(0, 85), figsize=(figure_width, 5)),
        ),
        (
            plot_average_unit_cost_and_price,
            dict(ylim=(0, 850), figsize=(figure_width, 4)),
        ),
        (
            plot_net_revenues,
            dict(ylim=(-46, 46), figsize=(figure_width, 5)),
        ),
    ]

In [ ]:
def make_images(network):
    imgs = []
    for plot_func, kwargs in list_plotting_tasks():
        try:
            fig, _ = plot_func(network, **kwargs)
            img = BytesIO()
            fig.tight_layout()
            px_per_inch = 85
            fig.savefig(img, format="png", dpi=px_per_inch * 2)
            plt.close()
            img.seek(0)
            img = widgets.Image(value=img.read(), format="png")
            width, height = kwargs["figsize"]
            img.layout = widgets.Layout(
                height=f"{px_per_inch*height}px",
                width=f"{px_per_inch*width}px",
                margin="5px 5px 5px 5px",
                border="solid 1.5px darkgrey",
            )
            imgs.append(img)
        except:
            logging.warning(f"Failed to create image with '{plot_func.__name__}'")
    return imgs


def make_widgets(values, images):
    header_layout = widgets.Layout(
        border="solid 1.5px darkgrey",
        margin="5px 5px 5px 5px",
        padding="5px 5px 5px 5px",
    )
    column_layout = widgets.Layout(
        margin="10px 5px 10px 5px",
        padding="5px 5px 5px 5px",
    )
    columns = []
    for key in ["previous", "current"]:
        html = f"""
            <center>
                <div><b>{key.upper()} CALCULATION</b></div>
                <div>p_min_pu: {values[key]['p_min_pu']:.0%}</div>
            </center>"""
        header = widgets.HTML(value=html)
        header.layout = header_layout
        column = widgets.VBox([header] + images[key])
        column.layout = column_layout
        columns.append(column)

    return widgets.HBox(columns)

In [ ]:
values = {"previous": None, "current": {"p_min_pu": default_p_min_pu}}
images = {"previous": None, "current": make_images(network)}


@interact_manual(
    p_min_pu=widgets.FloatSlider(default_p_min_pu, min=0, max=0.301, step=0.01)
)
def recalculate(p_min_pu):

    values["previous"] = values["current"]
    images["previous"] = images["current"]

    update_p_min_pu_of_public_power_plants(network, p_min_pu=p_min_pu)

    with suppress_stdout():
        optimise_network(network, params)

    logging.info("Generating result visualisations...")

    values["current"] = {"p_min_pu": p_min_pu}
    images["current"] = make_images(network)

    return make_widgets(values, images)